# Derivação da rede MLP (backpropagation)

![title](mlp-imagem.jpg)

## Variáveis

- $X$: matriz vertical $(n_0 \times 1)$ com os valores de entrada da rede neural para uma determinada amostrada;


- $W_1$, $W_2$ e $W_3$: matrizes de pesos entre as camadas neurais (vide figura) com dimensões $(\text{neurônios de saída} \times \text{neurônios/valores de entrada})$ de modo que suas colunas são preenchidos com os valores dos pesos conectados a cada entrada da rede neural:
$
\begin{bmatrix}
x1_1 & x2_1 & x3_1\\ 
x1_2 & x2_2 & x3_2
\end{bmatrix}
$


- $I_1$, $I_2$ e $I_3$: matriz de entrada ponderada nas camadas neurais;


- $Y_1$, $Y_2$ e $Y_3$: matriz de saída das camadas neurais após aplicação da função de ativação;


- $d$: matriz dos valores de saída $(m \times 1)$ desejados da rede neural.

## Função de ativação

Deve ser contínua e diferenciável em todo seu domínio. 

- **Sigmóide**:
$\frac{1}{1+e^{-x}}$

 - vantagens: pequena variação do gradiente, saídas entre 0 e 1 normalizando a saída cada neurônio, previsões claras à medida que se eleva o valor X tendo y os valores 0 ou 1.
 - desvantagens: Vanishing gradient (quando o gradiente fica tão pequeno que praticamente não altera os valores da matriz de pesos), saídas não centralizadas no zero, sem valores negativos. 
 
 
- **Tangente hiperbólica**:
$\tanh(x)$
    - vantagens: mesmas da sigmóide com a vantagem de ter valores negativos, zero e positivos para y
    - desvantagens: praticamente as mesmas da sigmóide
    
    
- **ReLu**:
$\max(0, x)$
    - vantagens: permite desativar certos neurônios por ter derivada zero para valores negativos de x. 
    - desvantagens: próximo do zero ou valores negativos há o problema da aprendizagem por backpropagation
    
    
- **Leaky ReLU**:
$f(x) = ax, x < 0$,
$f(x) = x, x \geqslant 0$
    - vantagens: resolve o problema de aprendizagem nos valores negativos e próximos a zero do ReLu
    - desvantagens: pode haver valores inconsistentes quando x < 0.
    obs: geralmente $a=0.01$


- **Swish**:
$\frac{x}{1+e^{-x}}$
    - vantagens: performance melhor que ReLu com mesmo esforço computacional

## Cálculo do erro

### Matriz erro na camada de saída
$\text{erro} = d - Y_3 = \begin{bmatrix}
e_1\\ 
e_2\\ 
...
\end{bmatrix}$

Com a matriz $\text{erro}$ de dimensões $(m \times 1)$ uma vez que há $m$ neurônios na saída da rede.

### Parâmetro do erro de desempenho local
Para uma determinada amostra:

$
E = \frac{e_1^2+e_2^2 + ... + e_m^2}{2}$

em que $e_m$ é o erro da $m$-ésima saída da rede neural. 

### Erro quadrático médio
Considerando todas as $p$ amostras, o erro quadrático médio é dado por

$E_M = \frac{(E_1 + E_2 + ... + E_p)}{p}$


## Implementação do algoritmo forward - MLP

1) Para uma dada amostra de valores da matriz de entrada $X$, tem-se que os vetores de entrada da primeira camada neural é dado por:

- $I_1 = W_1 \times X$ 

Dimensão: $(n_1 \times 1)$

2) Após isso, encontra-se a matriz da saída dos neurônios $(Y_1)$ com relação à camada 1, em que $f(x)$ representa o valor da função de ativação do neurônio:

- $Y_1 = f(I_1)$

Dimensão: $(n_1 \times 1)$

3) Para a entrada dos neurônios da segunda camada têm-se:

- $I_2 = W_2 \times Y_1$

Dimensão: $(n_2 \times 1)$

4) E as saídas da segunda camada:

- $Y_2 = f(I_2)$

Dimensão: $(n_2 \times 1)$

5) Analogamente para terceira camada:

- $I_3 = W_3 \times Y_2$

Dimensão: $(m \times 1)$

- $Y_3 = f(I_3)$

Dimensão: $(m \times 1)$

6) Cálculo do erro:

- $\text{erro} = d - Y_3$

- erro de desempenho local: $E = \frac{e_1^2+e_2^2 + ... + e_m^2}{2}$

## Implementação do algoritmo backforward - MLP

### Para camada de saída (3):

$\nabla E_3 = \frac{\partial E}{\partial W_3} = \frac{\partial E}{\partial Y_3}.\frac{\partial Y_3}{\partial I_3}.\frac{\partial I_3}{\partial W_3}$

- $\frac{\partial E}{\partial Y_3}$ (desempenho local)$ = \frac{2}{2}(d - Y_3)(-1) = -(d - Y_3)$

- $\frac{\partial Y_3}{\partial I_3} = f'(I_3)$

- $\frac{\partial I_3}{\partial W_3} = Y_2^T$

Assim, 

$\nabla E_3 = (-(d - Y_3)f'(I_3)) \times Y_2^T$

Como o gradiente tem direção crescente e queremos minimizar o erro, o ajuste de pesos da matrix $W_3$ tem que ser na direção oposta do gradiente de modo que:

$\Delta W_3 = -(-(d - Y_3)f'(I_3)) \times Y_2^T = \eta \delta_3 \times Y_2^T$

em que $\eta$ é a taxa de aprendizagem e $\delta_3 = (d - Y_3)f'(I_3)$ com dimensão $(m \times 1)$

Finalmente,

$W_{3final} = W_{3inicial} + \eta \delta_3 \times Y_2^T$

### Para camada escondida (2):

$\nabla E_2 = \frac{\partial E}{\partial W_2} = \frac{\partial E}{\partial Y_2}.\frac{\partial Y_2}{\partial I_2}.\frac{\partial I_2}{\partial W_2}$

 - $\frac{\partial E}{\partial Y_2} = \frac{\partial E}{\partial I_3} . \frac{\partial I_3}{\partial Y_2} = W_3^T \times \frac{\partial E}{\partial I_3} = W_3^T \times (-(d - Y_3).f'(I_3)) = -W_3^T \times \delta_3$
 - $\frac{\partial Y_2}{\partial I_2} = f'(I_2)$
 - $\frac{\partial I_2}{\partial W_2} = Y_1^T$
 
Então, 

$-\nabla E_2 = -\frac{\partial E}{\partial W_2} = -(-W_3^T \times \delta_3 . f'(I_2)) \times Y_1^T$

Definindo $\delta_2 = (W_3^T \times \delta_3). f'(I_2)$ com dimensão $(n_2 \times 1)$, tem-se, analogamente, para a matriz de peso $W_2$:

$W_{2final} = W_{2inicial} + \eta \delta_2 \times Y_1^T$

### Para camada inicial (1):

De forma análoga às camadas anteriores, tem-se:

$\delta_1 = (W_2^T \times \delta_2). f'(I_1)$ com dimensão $(n_1 \times 1)$

$W_{1final} = W_{1inicial} + \eta \delta_1 \times X^T$

#### OBSERVAÇÃO: As matrizes de pesos devem ser alteradas no mesmo instante, então, $\delta_1$ é em função do valor antigo de $W_2$ uma vez que tem que ser calculado ANTES que a atualização das matrizes de peso. 

## Generalização das equações

Sendo $L$ a $L$-ésima camada neural com $W_L$ é a matriz de pesos entre as camadas $L$ e $L+1$, as equações de correção dos pesos é generalizado por:

$W_{L_{final}} = W_{L_{inicial}} + \eta . \delta_L \times Y_{L-1}^T$

para $L \geqslant 1$ com $Y_0 = X$ tal que

$\delta_L = (W_{L+1}^T \times \delta_{L+1}).f'(I_L)$

sendo satisfeito para $L < \text{camada de saída}$, caso contrário

$\delta_L = (d - Y_L).f'(I_L)$

## Critério de parada da interação

A interação termina quando a diferença entre os erros quadráticos médios de duas épocas sucessivas for menor ou igual à precisão desejada:

$|E_{M_{atual}} - E_{M_{anterior}}| \leqslant \varepsilon$

ou então, se preferir, quando o erro quadrático médio atinge um valor desejado.

## Importância do bias

Em todo livro de redes neurais, considera-se um *bias*, ou seja, um valor, geralmente unitário, que é conectado as entradas das camadas neurais a partir de um peso. Sua função consiste em deslocar (*shift*) o gráfico resultante da função de ativação escolhido. 

Para exemplificar, seja uma rede neural com apenas uma entrada e um neurônio (sem bias) com função de ativação $y=x$, sua saída será dada por:

$y = f(xw) = xw$

Como pode-se perceber, se trata de uma reta que corta a origem no primeiro e terceiro quadrante (para $w>0$) e no segundo e quarto (para $w<0$). Para os valores de $x=1,2,3$ e $d=1,2,3$, respectivamente, poderíamos treinar a rede sem um bias sem maiores problemas, mas quando $x=1,2,3$ e $d=5,4,3$ ?

Nesta situação, é preciso que haja um deslocamento no gráfico, no eixo $y$, para satisfazer a equação $y = -d + 6$ e é exatamente nesse ponto que o bias se torna importante. No primeiro exemplo (tanh(x)) fiz um teste sem bias e foi possível convergir a rede, já no exemplo 2 usando a função de ativação *leaky ReLu* não obtive o mesmo sucesso, obtendo erros elevados na ordem de $7.77$. Realizando o mesmo teste com o bias, a *leaky ReLu* teve um desempenho até melhor convergindo rapidamente. Por esse motivo, é importante que sempre seja adicionado o bias nas entradas das camadas neurais e atualizado seu valor como mostrado no exemplo 3. 

# Algoritmo em Python

In [5]:
# Mostrar todos os output's
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Exemplo 1 - tanh(x)

In [4]:
import numpy as np
import pandas as pd


# Base de dados a ser treinada
# Entradas
x = pd.DataFrame(
	[[1],
	[2],
	[3]],
	columns=['valores x'])

# Valor desejado para saída
d = pd.DataFrame(
	[[5],
	[4],
	[3]],
	columns=['valores desejados'])

# Convertendo o dataframe em array e normalizando os valores desejados (para ficar entre 0 e +1)
x = x.to_numpy()
d = d/(1.05*d.max())
d = d.to_numpy()

# Derivada de tanh(x) = sech²(x) = 1 - (tanh(x))²
def df(x):
    y = 1 - np.power(np.tanh(x), 2)
    return y

#def rede_mlp(n, x, d, net, k, precisao):

# *** Construindo a rede de duas camadas ***

# número de neurônios na primeira camada
net=3
# taxa de aprendizagem
n = 0.1
# precisão do erro quadrático médio
precisao=0.01

# Inicializar matrizes de pesos com valores aleatórios distintos
w1 = np.random.rand(net,len(x[0]))
w2 = np.random.rand(1,net)

# Erro quadrático médio inicial e número de épocas
E_M=1
epocas=0

while E_M>precisao:
    E_M=0
    errofinal=0
    for i in range(0,len(x)):
        
        # FOWARD
        i1 = np.matmul(w1, x[i].reshape(len(x[i]),1))
        y1 = np.tanh(i1)

        i2 = np.matmul(w2, y1)
        y2 = np.tanh(i2)
        
        # erro com o valor desejado
        erro = d[i].reshape(len(d[i]),1) - y2
        
        # BACKPROPAGATION
        delta_2 = erro*df(i2)
        delta_1 = (np.matmul(w2.T, delta_2))*df(i1)
        
        w2 = w2 + n*(np.matmul(delta_2, y1.reshape(1, net)))
        w1 = w1 + n*(np.matmul(delta_1, x[i].reshape(1, len(x[i]))))
        
        errofinal = errofinal + 0.5*erro**2
        
    E_M = errofinal/len(x)
    epocas+=1
    print(E_M)
    
    
def conferir_peso(w1, w2, entrada, fator_multiplicativo):
    i1 = np.matmul(w1, np.array(entrada).T)
    y1 = np.tanh(i1)
    i2 = np.matmul(w2, y1)
    y2 = np.tanh(i2)
    y2 = y2*fator_multiplicativo
    return(y2)

# Conferir o número de épocas para a convergência e o valor encontrado para alguma das entradas
print('\n')
valor = conferir_peso(w1, w2, [2], 1.05*5)
print(valor)
print(epocas)




[3.89973329]
490


## Exemplo 2 - Leaky ReLu (sem bias)

In [3]:
import numpy as np
import pandas as pd


# Base de dados a ser treinada
x = pd.DataFrame(
	[[1],
	[2],
	[3]],
	columns=['valores x'])

d = pd.DataFrame(
	[[5],
	[4],
	[3]],
	columns=['valores desejados'])

# Convertendo o dataframe em array e normalizando os valores desejados para ficar entre 0 e +1.
x = x.to_numpy()
# d = d/(1.05*d.max())
d = d.to_numpy()


# Derivada de tanh(x) = sech²(x) = 1 - (tanh(x))²
def df(x):
    x = np.array(x)
    x[x<=0] = 0.01
    x[x>0] = 1
    return x

def f(x):
    return(np.where(x > 0, x, x * 0.01))
#     return(np.maximum(x, 0))

    

#def rede_mlp(n, x, d, net, k, precisao):

# Construindo a rede de duas camadas 
# net = número de neurônios na primeira camada
# n = taxa de aprendizagem
# precisao = precisão do erro quadrático médio
net=3
n = 1e-4
precisao=0.0001
w1 = np.random.rand(net,len(x[0]))
w2 = np.random.rand(1,net)
E_M=20
epocas=0

while E_M>precisao:
    E_M=0
    errofinal=0
    for i in range(0,len(x)):
        
        # FOWARD
        i1 = np.matmul(w1, x[i].reshape(len(x[i]),1))
        y1 = f(i1)
        
        

        i2 = np.matmul(w2, y1)
        y2 = f(i2)
        
        
        # erro com o valor desejado
        erro = d[i].reshape(len(d[i]),1) - y2
        
        
        # BACKPROPAGATION
        delta_2 = erro*df(i2)
        delta_1 = (np.matmul(w2.T, delta_2))*df(i1)
        
        w2 = w2 + n*(np.matmul(delta_2, y1.reshape(1, net)))
        w1 = w1 + n*(np.matmul(delta_1, x[i].reshape(1, len(x[i]))))
        
          
        errofinal = errofinal + 0.5*erro**2
        
    #E_M = errofinal/len(x)
    E_M = errofinal
    epocas+=1
    print(E_M)
    
    
def conferir_peso(w1, w2, entrada, fator_multiplicativo):
    i1 = np.matmul(w1, np.array(entrada).T)
    y1 = np.tanh(i1)

    i2 = np.matmul(w2, y1)
    y2 = np.tanh(i2)
    y2 = y2*fator_multiplicativo
    return(y2)

#1.05*5
valor = conferir_peso(w1, w2, [2], 1)
valor
print(epocas)

KeyboardInterrupt: 

## Exemplo 3 - Leaky ReLu (com bias)

In [2]:
import numpy as np
import pandas as pd


# Base de dados a ser treinada
x = pd.DataFrame(
    [[1],
    [2],
    [3]],
    columns=['valores x'])

d = pd.DataFrame(
    [[5],
    [4],
    [3]],
    columns=['valores desejados'])

# Convertendo o dataframe em array e normalizando os valores desejados para ficar entre 0 e +1.
x = x.to_numpy()
d = d.to_numpy()


def df(x):
    return(np.where(np.array(x) <= 0, 0.01, 1))

def f(x):
    return(np.where(np.array(x) > 0, x, x * 0.01))


#def rede_mlp(n, x, d, net, k, precisao):

# Construindo a rede de duas camadas 
# net = número de neurônios na primeira camada
# n = taxa de aprendizagem
# precisao = precisão do erro quadrático médio
net=3
n = 1e-3
precisao=1e-5
w1 = np.random.rand(net,len(x[0]))
bias1 = np.random.rand()

w2 = np.random.rand(1,net)
bias2 = np.random.rand()

E_M=20
epocas=0

while E_M>precisao:
    E_M=0
    errofinal=0
    for i in range(0,len(x)):

        # FOWARD
        i1 = np.matmul(w1, x[i].reshape(len(x[i]),1)) + bias1
        y1 = f(i1)


        i2 = np.matmul(w2, y1) + bias2
        y2 = f(i2)


        # erro com o valor desejado
        erro = d[i].reshape(len(d[i]),1) - y2


        # BACKPROPAGATION
        delta_2 = erro*df(i2)
        delta_1 = (np.matmul(w2.T, delta_2))*df(i1)
        bias2 += n * delta_2
        bias1 += n * delta_1
        
        w2 = w2 + n*(np.matmul(delta_2, y1.reshape(1, net)))
        w1 = w1 + n*(np.matmul(delta_1, x[i].reshape(1, len(x[i]))))


        errofinal = errofinal + 0.5*erro**2

    #E_M = errofinal/len(x)
    E_M = errofinal
    epocas+=1
    print(E_M)